# Evidencia de Funcionalidad: Agente con Reconocimiento Visual

Este notebook demuestra la integración final donde el **Planner (Search Service)** es capaz de utilizar la nueva herramienta de **Reconocimiento de Imagen**.

Se simula el flujo de un agente que decide usar la herramienta correcta basándose en la petición del usuario.


In [ ]:
import sys
import os
from unittest.mock import MagicMock, AsyncMock

# Setup paths
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from business_backend.services.search_service import SearchService, SearchResult
from business_backend.ml.serving.inference_service import InferenceService, PredictionResult
from business_backend.ml.models.registry import ModelRegistry, BaseModel
from business_backend.llm.provider import LLMProvider
from business_backend.llm.tools.image_recognition_tool import ImageRecognitionTool

## 1. Configuración de Mocks

Para esta demostración aislada, simulamos:
1.  **ProductService**: Base de datos de productos.
2.  **InferenceService**: El servicio ML que creamos anteriormente.
3.  **LLMProvider**: Simulamos la decisión del LLM de llamar a una tool.

Esto prueba la lógica de orquestación en `SearchService` sin gastar tokens de OpenAI.

In [ ]:
# 1. Mock Product Service
product_service_mock = AsyncMock()
product_service_mock.search_by_name.return_value = [] # Default empty

# 2. Mock Inference Service (Using logic similar to real one)
inference_service_mock = AsyncMock(spec=InferenceService)
inference_service_mock.predict.return_value = PredictionResult(
    model_name="product_classifier",
    prediction="Leche Entera 1L",
    confidence=0.99
)

# 3. Mock LLM Provider that simulates tool calling
llm_provider_mock = MagicMock(spec=LLMProvider)
mock_model = AsyncMock()
llm_provider_mock.bind_tools.return_value = mock_model

# Simular respuesta del LLM diciendo "Hey, usa la herramienta image_recognition con esta imagen"
tool_call_response = MagicMock()
tool_call_response.tool_calls = [
    {
        "name": "image_recognition",
        "args": {"image_path": "/path/to/leche.jpg"},
        "id": "call_123"
    }
]
tool_call_response.content = "Voy a analizar la imagen."

# Simular respuesta final después de usar la herramienta
final_response = MagicMock()
final_response.content = "He analizado la imagen y veo que es 'Leche Entera 1L'."
final_response.tool_calls = None

# Configurar el side_effect para devolver primero la llamada a tool, luego la respuesta final
mock_model.ainvoke.side_effect = [tool_call_response, final_response]

## 2. Inyección de Dependencias

Aquí es donde ocurre la magia de la integración. Inyectamos, además del servicio de productos, el `inference_service` al `SearchService`.


In [ ]:
search_service = SearchService(
    llm_provider=llm_provider_mock,
    product_service=product_service_mock,
    inference_service=inference_service_mock  # <--- Integración clave
)

print("SearchService inicializado.")
if search_service.image_tool:
    print("✅ Herramienta de Imagen registrada correctamente en el servicio.")
else:
    print("❌ Fallo al registrar herramienta de imagen.")

## 3. Ejecución del Agente

Simulamos un usuario preguntando: *"¿Qué es esto?"* enviando una imagen.


In [ ]:
query = "¿Qué producto es este? /path/to/leche.jpg"

result = await search_service.semantic_search(query)

print("\n--- Resultado Final del Agente ---\n")
print(f"Respuesta: {result.answer}")

## 4. Verificación de Llamadas Internas

Verificamos que el Agente realmente llamó al servicio de ML por debajo.

In [ ]:
# Verificar que se llamó al InferenceService
inference_service_mock.predict.assert_called_once()
call_args = inference_service_mock.predict.call_args

print("Validación de Integración:")
print(f"✅ Se llamó a 'InferenceService.predict'? {inference_service_mock.predict.called}")
print(f"✅ Argumentos usados: {call_args}")

## Conclusión

Esta evidencia demuestra que el **Planner** ahora tiene capacidad **Multimodal**. 
Si el usuario pregunta sobre una imagen, el sistema:
1. Detecta la intención.
2. Invoca `image_recognition_tool`.
3. La tool usa `InferenceService` para clasificar.
4. El LLM interpreta la clasificación y responde al usuario.